# Assignment 3 part 2

#### Get dataframe from wiki

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(wiki_link)
soup = BeautifulSoup(r.text, 'html.parser')
data = []
na='Not assigned'
table = soup.table
table_body = table.find('tbody')

headers=[h.get_text() for h in table_body.find_all('th')]
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols)>1 and cols[1]!=na:
        data.append([ele for ele in cols if ele]) #
    
headers[-1]='Neighbourhood'
#data[0]=headers
del data[0]
df = pd.DataFrame(data, columns=headers)
df.Neighbourhood.replace('Not assigned',df.Borough,inplace=True)

df['Neighbourhood'] = df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ','.join(x))
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)

df.head()


,index,Postcode,Borough,Neighbourhood
0,0,M4A,North York,Victoria Village
1,1,M5A,Downtown Toronto,"Harbourfront,Regent Park"
2,3,M6A,North York,"Lawrence Heights,Lawrence Manor"
3,5,M7A,Queen's Park,Queen's Park
4,6,M9A,Etobicoke,Islington Avenue


#### Try geocoder

In [13]:
!pip install geocoder

import geocoder

    100% |████████████████████████████████| 102kB 6.1MB/s ta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

### Try to use Geocoder but it do not response

In [ ]:
latitude=[]
longitude=[]

for postal_code in df['Postcode']:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    counter=0
    while(lat_lng_coords is None and counter<100):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        counter+=1
 
    if lat_lng_coords is None:
        lat_lng_coords[0]=0
        lat_lng_coords[1]=1
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
len(latitude)

df['latitude']=pd.Series(latitude)
df['longitude']=pd.Series(longitude)

df.head()

### Data frame from prepared file

In [ ]:
final_df=pd.read_csv('http://cocl.us/Geospatial_data')
final_df.head()